## Intro to Conversational AI and Rasa

In [ ]:
!pip install nest_asyncio
!pip install rasa

### Start the event loop

Running asynchronous Rasa code in Jupyter Notebooks requires an extra requirement, since Jupyter Notebooks already run on event loops. Run the cell below to start the event loop:

In [ ]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

### Create a new project and train an example assistant

Create a new project called **rasabot**. The function below will create a new project with example training data and most important files. It replicates the behaviour of Rasa CLI function **rasa init**.


In [ ]:
import rasa
from rasa.cli.scaffold import create_initial_project
import os

project = "rasabot"
create_initial_project(project)

# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

# define the project files
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"

#train the example assistant
model_path = rasa.train(domain, config, [training_files], output)

### Have a chat with an assistant

The function above creates an assistant called moodbot which asks how you feel and if you are sad it tries to cheer your up with a picture of a cute tiger cub. You can test an assistant using the functions below. To stop the chat type '/stop' in a chat window.
The code cell below replicates the behaviour of the Rasa CLI function **rasa shell**.

In [ ]:
from rasa.jupyter import chat
chat(model_path)

## Creating the NLU training data

Now, we learn about the format of the NLU training data and create the necessary intent for our assistant. By running the fell below, you will update the training dataset with the new created intents. You are encouraged to add more training examples to existing intents or create completely new ones.

In [ ]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:inform
- [Sitka](location)
- [Juneau](location)
- [Virginia](location)
- [Cusseta](location)
- [Chicago](location)
- [Tuscon](location)
- [Columbus](location)
- [San Francisco](location)

## intent:search_provider
- I need a [hospital](facility_type)
- find me a nearby [hospital](facility_type)
- show me [home health agencies](facility_type)
- [hospital](facility_type)
- find me a nearby [hospital](facility_type) in [San Francisco](location)
- I need a [home health agency](facility_type)

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad
"""

%store nlu_md >data/nlu.md

## Training the NLU model: pre-configured pipelines

Now we talk about the pre-configured Rasa NLU training pipelines.

### Updating the NLU configuration pipeline
An NLU training pipeline can be specified by editing a file called config.yml in your project directory. By editing the cell below you can change the pre-configured pipelines from supervised_embeddings to pretrained_embeddings_spacy and test the performance.

More about pipelines [in the blog] (https://blog.rasa.com/the-rasa-masterclass-handbook-episode-3/)

In [ ]:
config_yml = """

language: en
pipeline: supervised_embeddings


policies:
  - name: MemoizationPolicy
  - name: KerasPolicy
  - name: MappingPolicy

"""

%store config_yml >config.yml

### Train and test the NLU model
First, make sure to start the event loop for jupyter notebook.


In [ ]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

Specify the model configuration, data files as well as the directory to store the trained model.

In [ ]:
import rasa
from rasa.train import train_nlu

config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"

Train the model:

In [ ]:
mod = train_nlu(config, training_files, output)

## Intro to dialogue management with Rasa
we are looking into the Dialogue management with Rasa - understanding the structure of training data (stories) and creating new examples.

Dialogue management training data (stories) correspond to actual conversations between the user and an assistant where user inputs are expressed as corresponding intents while the responses of the assistant are expressed as corresponding actions.

Create some training stories for the assistant. Run the cell below to update the stories in your stories.md file. You are encouraged to write more stories with different dialogue turns.

In [ ]:
stories_md = """

## hospital search happy path
* greet
  - utter_how_can_i_help
* search_provider{"facility_type":"hospital", "location": "San Francisco"}
  - action_facility_search
* thanks
  - utter_goodbye

## hospital search + location
* greet
  - utter_how_can_i_help
* search_provider{"facility_type":"hospital"}
  - utter_ask_location
* inform{"location":"San Francisco"}
  - action_facility_search
* thanks
  - utter_goodbye

## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

"""
%store stories_md >data/stories.md

## Domain, custom actions and slots

Cover the reamining components of the Rasa dialogue management models - domain, custom actions and slots.

### Define a domain
Update the domain with all intents, entities, actions and template for the assistant:



In [ ]:
domain_yml = """
intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - inform
  - search_provider
  - thanks

entities:
  - location
  - facility_type

slots:
  location:
    type: text
  facility_type:
    type: text
  address:
    type: unfeaturized

actions:
- utter_greet
- utter_cheer_up
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_how_can_i_help
- utter_ask_location
- action_facility_search

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_how_can_i_help:
  - text: "Hello. I am a medicare locator. I can help you find nearset hospital. How can I help?"
  - text: "Hi. My name is medicare locator. How can I help you today?"

  utter_ask_location:
  - text: "Can you provide your location please?"
  - text: "To find the nearest {facility_type} I need your address."
"""

%store domain_yml >domain.yml

### Implement the custom action
In the episode we also implemented a simple custom actions which looks for a specified facility based on user's preferences and sets the returne address as a slot.

In [ ]:
action_py = """

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet


class ActionFacilitySearch(Action):

    def name(self) -> Text:
        return "action_facility_search"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        facility = tracker.get_slot("facility_type")
        address = "300 Hyde St, San Francisco"
        dispatcher.utter_message("Here is the address of the {}:{}".format(facility, address))

        return [SlotSet("address", address)]
        
"""        
%store action_py >actions.py

### Updating the stories
Training stories should be updated to reflect the SlotSet events which are executed in specific custom ations:

In [ ]:
stories_md = """
## hospital search happy path
* greet
  - utter_how_can_i_help
* search_provider{"facility_type":"hospital", "location": "San Francisco"}
  - action_facility_search
  - slot{"address":"300 Hyde St, San Francisco"}
* thanks
  - utter_goodbye

## hospital search + location
* greet
  - utter_how_can_i_help
* search_provider{"facility_type":"hospital"}
  - utter_ask_location
* inform{"location":"San Francisco"}
  - action_facility_search
  - slot{"address":"300 Hyde St, San Francisco"}
* thanks
  - utter_goodbye

## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

"""        
%store stories_md >data/stories.md


### Train the model
With training data, domain, custom actions in place we can train and test our assistant to see how it works with NLU and dialogue management models in place.

In [ ]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

In [ ]:
import rasa

# define the project files
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"

#train the example assistant
model_path = rasa.train(domain, config, [training_files], output)

Test the assistant:

In [ ]:
from rasa.jupyter import chat
chat(model_path)